In [1]:
import time

import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchvision import utils
from collections import Counter

from basic_fcn import *
from dataloader import *
from utils import *

import matplotlib.pyplot as plt
from torchvision import transforms 
import PIL

## Flip method

In [ ]:
writer = SummaryWriter()

train_dataset = IddDataset(csv_file='train.csv', transforms_='flip')
val_dataset = IddDataset(csv_file='val.csv')
test_dataset = IddDataset(csv_file='test.csv')


train_loader = DataLoader(dataset=train_dataset, batch_size=4, num_workers=8, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=4, num_workers=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, num_workers=8, shuffle=False)

weighted = 'False'


def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.xavier_uniform(m.bias.data)


nSamples = [2198453584,519609102,28929892,126362539,58804972,59032134,94293190,2569952,101519794,163659019,105075839,47400301,30998126,133835645,135950687,41339482,15989829,104795610,8062798,450973,94820222,341805135,557016609,71159311,1465165566,1823922767,2775322]
# nSamples = [0.263799707945899,0.0623496126328182,0.00347139330848224,0.0151626584823554,0.00705620284743149,0.00708346076622156,0.0113145513575178,0.000308377030094705,0.0121816954439407,0.0196379863232593,0.0126083970305785,0.00568771869979480,0.00371956753837482,0.0160593811580563,0.0163131720344951,0.00496046100659214,0.00191867240273055,0.0125747714271562,0.000967481766777560,5.41137400203969e-05,0.0113777916682026,0.0410143272724236,0.0668382629702202,0.00853864079553448,0.175810056312377,0.218858517983828,0.000333020054816781]

normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
normedWeights = torch.FloatTensor(normedWeights).cuda()

epochs = 20
if weighted:
    criterion = nn.CrossEntropyLoss(weight = normedWeights)
else:
    criterion = nn.CrossEntropyLoss()
    
fcn_model = FCN(n_class=n_class)
# fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=0.0001)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()


def train():
    last_iter = 0
    for epoch in range(epochs):
        fcn_model.train()
        ts = time.time()
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(train_loader):
            optimizer.zero_grad()
            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X, Y

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()
            num_iter += 1
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
                writer.add_scalar('Loss/train_batch', loss.item(), last_iter)
            last_iter += 1

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'flip_model')
        writer.add_scalar('Loss/train_epoch', epoch_loss / num_iter, epoch)
        val(epoch)


def val(epoch):
    print(f'Start val epoch{epoch}')
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(val_loader):
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            ts = time.time()
            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            valoutputs = fcn_model(inputs)
            valloss = criterion(valoutputs, Y)
            epoch_loss += valloss.item()
            num_iter += 1
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)
        writer.add_scalar('Loss/Val_epoch', epoch_loss / num_iter, epoch)
    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Val{}, pixel acc {}, avg iou {}, time elapsed {}".format(epoch, pixel_acc, np.mean(iou), time.time() - ts))

    writer.add_scalar('Pixel Accuracy/Val', pixel_acc, epoch)
    writer.add_scalar('IOU/Average/Val', np.mean(iou), epoch)
    writer.add_scalar('IOU/Road/Val', iou[0], epoch)
    writer.add_scalar('IOU/Sidewalk/Val', iou[2], epoch)
    writer.add_scalar('IOU/Car/Val', iou[9], epoch)
    writer.add_scalar('IOU/Billboard/Val', iou[17], epoch)
    writer.add_scalar('IOU/Sky/Val', iou[25], epoch)


def test():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)

    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Test, pixel acc {}, avg iou {}, time elapsed {}".format(pixel_acc, np.mean(iou), time.time() - ts))
    print(f"ious result: {iou}")

def test_visualization():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            break
    imgs = []
    for i in pred[0]:
        for j in i:
            imgs.append(labels[j][2])
    imgs = np.asarray(imgs).reshape(pred.shape[1], pred.shape[2],3)
    outputimg = PIL.Image.fromarray(np.array(imgs, dtype=np.uint8))
    plt.axis('off')
    plt.imshow(outputimg)
    plt.title('Output Image')
    plt.show()      
    

train()
test_visualization()
writer.close()

epoch0, iter0, loss: 3.502537727355957
epoch0, iter10, loss: 3.18190336227417
epoch0, iter20, loss: 2.7764647006988525
epoch0, iter30, loss: 2.471342086791992
epoch0, iter40, loss: 2.416844606399536
epoch0, iter50, loss: 2.2714836597442627
epoch0, iter60, loss: 2.268047571182251
epoch0, iter70, loss: 1.9197691679000854
epoch0, iter80, loss: 2.0814831256866455
epoch0, iter90, loss: 1.8322590589523315
epoch0, iter100, loss: 2.009024143218994
epoch0, iter110, loss: 1.7190982103347778
epoch0, iter120, loss: 1.6226491928100586
epoch0, iter130, loss: 1.816345453262329
epoch0, iter140, loss: 1.609078049659729
epoch0, iter150, loss: 1.837506651878357
epoch0, iter160, loss: 2.372776985168457
epoch0, iter170, loss: 2.112110137939453
epoch0, iter180, loss: 1.7244508266448975
epoch0, iter190, loss: 1.5762404203414917
epoch0, iter200, loss: 1.4730520248413086
epoch0, iter210, loss: 1.6838794946670532
epoch0, iter220, loss: 1.5130113363265991
epoch0, iter230, loss: 1.9254863262176514
epoch0, iter240

## Rotate method

In [ ]:
writer_1 = SummaryWriter()

train_dataset = IddDataset(csv_file='train.csv', transforms_='rotate')

train_loader = DataLoader(dataset=train_dataset, batch_size=4, num_workers=8, shuffle=True)

weighted = 'False'


def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.xavier_uniform(m.bias.data)


nSamples = [2198453584,519609102,28929892,126362539,58804972,59032134,94293190,2569952,101519794,163659019,105075839,47400301,30998126,133835645,135950687,41339482,15989829,104795610,8062798,450973,94820222,341805135,557016609,71159311,1465165566,1823922767,2775322]
# nSamples = [0.263799707945899,0.0623496126328182,0.00347139330848224,0.0151626584823554,0.00705620284743149,0.00708346076622156,0.0113145513575178,0.000308377030094705,0.0121816954439407,0.0196379863232593,0.0126083970305785,0.00568771869979480,0.00371956753837482,0.0160593811580563,0.0163131720344951,0.00496046100659214,0.00191867240273055,0.0125747714271562,0.000967481766777560,5.41137400203969e-05,0.0113777916682026,0.0410143272724236,0.0668382629702202,0.00853864079553448,0.175810056312377,0.218858517983828,0.000333020054816781]

normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
normedWeights = torch.FloatTensor(normedWeights).cuda()

epochs = 20
if weighted:
    criterion = nn.CrossEntropyLoss(weight = normedWeights)
else:
    criterion = nn.CrossEntropyLoss()
    
fcn_model = FCN(n_class=n_class)
# fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=0.0001)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()


def train():
    last_iter = 0
    for epoch in range(epochs):
        fcn_model.train()
        ts = time.time()
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(train_loader):
            optimizer.zero_grad()
            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X, Y

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()
            num_iter += 1
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
                writer_1.add_scalar('Loss/train_batch', loss.item(), last_iter)
            last_iter += 1

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'rotate_model')
        writer_1.add_scalar('Loss/train_epoch', epoch_loss / num_iter, epoch)
        val(epoch)


def val(epoch):
    print(f'Start val epoch{epoch}')
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(val_loader):
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            ts = time.time()
            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            valoutputs = fcn_model(inputs)
            valloss = criterion(valoutputs, Y)
            epoch_loss += valloss.item()
            num_iter += 1
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)
        writer_1.add_scalar('Loss/Val_epoch', epoch_loss / num_iter, epoch)
    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Val{}, pixel acc {}, avg iou {}, time elapsed {}".format(epoch, pixel_acc, np.mean(iou), time.time() - ts))

    writer_1.add_scalar('Pixel Accuracy/Val', pixel_acc, epoch)
    writer_1.add_scalar('IOU/Average/Val', np.mean(iou), epoch)
    writer_1.add_scalar('IOU/Road/Val', iou[0], epoch)
    writer_1.add_scalar('IOU/Sidewalk/Val', iou[2], epoch)
    writer_1.add_scalar('IOU/Car/Val', iou[9], epoch)
    writer_1.add_scalar('IOU/Billboard/Val', iou[17], epoch)
    writer_1.add_scalar('IOU/Sky/Val', iou[25], epoch)


def test():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)

    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Test, pixel acc {}, avg iou {}, time elapsed {}".format(pixel_acc, np.mean(iou), time.time() - ts))
    print(f"ious result: {iou}")

def test_visualization():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            break
    imgs = []
    for i in pred[0]:
        for j in i:
            imgs.append(labels[j][2])
    imgs = np.asarray(imgs).reshape(pred.shape[1], pred.shape[2],3)
    outputimg = PIL.Image.fromarray(np.array(imgs, dtype=np.uint8))
    plt.axis('off')
    plt.imshow(outputimg)
    plt.title('Output Image')
    plt.show()      
    

train()
test_visualization()
writer_1.close()

## Rotate and weighted loss

In [ ]:
writer_2 = SummaryWriter()

train_dataset = IddDataset(csv_file='train.csv', transforms_='rotate')
val_dataset = IddDataset(csv_file='val.csv')
test_dataset = IddDataset(csv_file='test.csv')


train_loader = DataLoader(dataset=train_dataset, batch_size=4, num_workers=8, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=4, num_workers=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, num_workers=8, shuffle=False)

weighted = 'True'


def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.xavier_uniform(m.bias.data)


nSamples = [2198453584,519609102,28929892,126362539,58804972,59032134,94293190,2569952,101519794,163659019,105075839,47400301,30998126,133835645,135950687,41339482,15989829,104795610,8062798,450973,94820222,341805135,557016609,71159311,1465165566,1823922767,2775322]
# nSamples = [0.263799707945899,0.0623496126328182,0.00347139330848224,0.0151626584823554,0.00705620284743149,0.00708346076622156,0.0113145513575178,0.000308377030094705,0.0121816954439407,0.0196379863232593,0.0126083970305785,0.00568771869979480,0.00371956753837482,0.0160593811580563,0.0163131720344951,0.00496046100659214,0.00191867240273055,0.0125747714271562,0.000967481766777560,5.41137400203969e-05,0.0113777916682026,0.0410143272724236,0.0668382629702202,0.00853864079553448,0.175810056312377,0.218858517983828,0.000333020054816781]

normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
normedWeights = torch.FloatTensor(normedWeights).cuda()

epochs = 20
if weighted:
    criterion = nn.CrossEntropyLoss(weight = normedWeights)
else:
    criterion = nn.CrossEntropyLoss()
    
fcn_model = FCN(n_class=n_class)
# fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=0.0001)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()


def train():
    last_iter = 0
    for epoch in range(epochs):
        fcn_model.train()
        ts = time.time()
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(train_loader):
            optimizer.zero_grad()
            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X, Y

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()
            num_iter += 1
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
                writer_2.add_scalar('Loss/train_batch', loss.item(), last_iter)
            last_iter += 1

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'weighted_model')
        writer_2.add_scalar('Loss/train_epoch', epoch_loss / num_iter, epoch)
        val(epoch)


def val(epoch):
    print(f'Start val epoch{epoch}')
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        epoch_loss = 0
        num_iter = 0
        for iter, (X, Y) in enumerate(val_loader):
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            ts = time.time()
            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            valoutputs = fcn_model(inputs)
            valloss = criterion(valoutputs, Y)
            epoch_loss += valloss.item()
            num_iter += 1
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)
        writer_2.add_scalar('Loss/Val_epoch', epoch_loss / num_iter, epoch)
    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Val{}, pixel acc {}, avg iou {}, time elapsed {}".format(epoch, pixel_acc, np.mean(iou), time.time() - ts))

    writer_2.add_scalar('Pixel Accuracy/Val', pixel_acc, epoch)
    writer_2.add_scalar('IOU/Average/Val', np.mean(iou), epoch)
    writer_2.add_scalar('IOU/Road/Val', iou[0], epoch)
    writer_2.add_scalar('IOU/Sidewalk/Val', iou[2], epoch)
    writer_2.add_scalar('IOU/Car/Val', iou[9], epoch)
    writer_2.add_scalar('IOU/Billboard/Val', iou[17], epoch)
    writer_2.add_scalar('IOU/Sky/Val', iou[25], epoch)


def test():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            iouMetric.update(pred, Y)
            pixelAccMetric.update(pred, Y)

    pixel_acc = pixelAccMetric.result()
    iou = iouMetric.result()
    print("Test, pixel acc {}, avg iou {}, time elapsed {}".format(pixel_acc, np.mean(iou), time.time() - ts))
    print(f"ious result: {iou}")

def test_visualization():
    iouMetric = IOUMetric()
    pixelAccMetric = PixelAccMetric()
    fcn_model.eval()  # Don't forget to put in eval mode !
    ts = time.time()
    with torch.no_grad():
        for X, Y in test_loader:
            if use_gpu:
                inputs = X.cuda()
                Y = Y.cuda()
            else:
                inputs = X

            outputs = F.log_softmax(fcn_model(inputs), dim=1)
            _, pred = torch.max(outputs, dim=1)
            break
    imgs = []
    for i in pred[0]:
        for j in i:
            imgs.append(labels[j][2])
    imgs = np.asarray(imgs).reshape(pred.shape[1], pred.shape[2],3)
    outputimg = PIL.Image.fromarray(np.array(imgs, dtype=np.uint8))
    plt.axis('off')
    plt.imshow(outputimg)
    plt.title('Output Image')
    plt.show()      
    

train()
test_visualization()
writer_2.close()

In [5]:
test()

Test, pixel acc 0.679888277473058, avg iou 0.23375702941162754, time elapsed 9.982284545898438
ious result: [0.8077169225025898, 0.3058032604528347, 0.01380392028051274, 0.1313924881582949, 0.08322177449193076, 0.2841225879979374, 0.29567537472942035, 0.0, 0.3068296967960644, 0.40582778905762484, 0.17221215444030755, 0.0938017643494419, 0.03317397687335097, 0.3360090335367613, 0.11481311355144926, 0.1949977515355842, 0.10559567542237433, 0.06597157985143966, 0.07399986473233912, 0.0, 0.2462615057909274, 0.11046307614775096, 0.2277781526156604, 0.11545668822020735, 0.6980206800171436, 0.854733933150368]


## Rotate method without normedweights

In [6]:
weighted = 'False'
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.xavier_uniform(m.bias.data)


nSamples = [10, 10, 1, 1, 1, 5, 5, 1, 10, 5, 5, 1, 1, 5, 1, 1, 1, 5, 1, 1, 5, 5, 10, 1, 5, 10, 1]
normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
normedWeights = torch.FloatTensor(normedWeights).cuda()

epochs = 20
if weighted:
    criterion = nn.CrossEntropyLoss(weight = normedWeights)
else:
    criterion = nn.CrossEntropyLoss()
    
fcn_model = FCN(n_class=n_class)
# fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=0.0001)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
train()
test()

epoch0, iter0, loss: 3.389866828918457
epoch0, iter10, loss: 3.2945544719696045
epoch0, iter20, loss: 2.9168567657470703
epoch0, iter30, loss: 2.765124797821045
epoch0, iter40, loss: 2.647425889968872
epoch0, iter50, loss: 2.3501851558685303
epoch0, iter60, loss: 2.561572551727295
epoch0, iter70, loss: 2.161515235900879
epoch0, iter80, loss: 2.1175289154052734
epoch0, iter90, loss: 2.2770071029663086
epoch0, iter100, loss: 2.076861619949341
epoch0, iter110, loss: 1.9426995515823364
epoch0, iter120, loss: 2.2393689155578613
epoch0, iter130, loss: 1.9368155002593994
epoch0, iter140, loss: 2.1262779235839844
epoch0, iter150, loss: 2.3373448848724365
epoch0, iter160, loss: 1.7918237447738647
epoch0, iter170, loss: 2.5723624229431152
epoch0, iter180, loss: 2.2259433269500732
epoch0, iter190, loss: 1.9662022590637207
epoch0, iter200, loss: 2.1064934730529785
epoch0, iter210, loss: 2.7087676525115967
epoch0, iter220, loss: 2.2082650661468506
epoch0, iter230, loss: 2.0763845443725586
epoch0, i

## Flip method without normedweights

In [7]:
train_dataset = IddDataset(csv_file='train.csv', transforms_='flip')
val_dataset = IddDataset(csv_file='val.csv')
test_dataset = IddDataset(csv_file='test.csv')


train_loader = DataLoader(dataset=train_dataset, batch_size=4, num_workers=8, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=4, num_workers=8, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, num_workers=8, shuffle=False)
weighted = 'False'
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.xavier_uniform(m.bias.data)


nSamples = [10, 10, 1, 1, 1, 5, 5, 1, 10, 5, 5, 1, 1, 5, 1, 1, 1, 5, 1, 1, 5, 5, 10, 1, 5, 10, 1]
normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
normedWeights = torch.FloatTensor(normedWeights).cuda()

epochs = 20
if weighted:
    criterion = nn.CrossEntropyLoss(weight = normedWeights)
else:
    criterion = nn.CrossEntropyLoss()
    
fcn_model = FCN(n_class=n_class)
# fcn_model.apply(init_weights)

optimizer = optim.Adam(fcn_model.parameters(), lr=0.0001)

use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
train()
test()

epoch0, iter0, loss: 3.4625840187072754
epoch0, iter10, loss: 3.3212528228759766
epoch0, iter20, loss: 3.044649600982666
epoch0, iter30, loss: 3.029639959335327
epoch0, iter40, loss: 2.7461349964141846
epoch0, iter50, loss: 2.6223530769348145
epoch0, iter60, loss: 2.4759373664855957
epoch0, iter70, loss: 2.3229784965515137
epoch0, iter80, loss: 2.407968521118164
epoch0, iter90, loss: 2.060825824737549
epoch0, iter100, loss: 2.463724136352539
epoch0, iter110, loss: 1.816458821296692
epoch0, iter120, loss: 2.17069673538208
epoch0, iter130, loss: 1.9615124464035034
epoch0, iter140, loss: 2.1075034141540527
epoch0, iter150, loss: 2.2961676120758057
epoch0, iter160, loss: 1.7841373682022095
epoch0, iter170, loss: 1.8152070045471191
epoch0, iter180, loss: 1.9172276258468628
epoch0, iter190, loss: 2.240150213241577
epoch0, iter200, loss: 2.177438735961914
epoch0, iter210, loss: 2.172213554382324
epoch0, iter220, loss: 1.572794795036316
epoch0, iter230, loss: 1.6080822944641113
epoch0, iter240

KeyboardInterrupt: 